In [1]:
import torch
from PIL import Image
from transformers import AutoProcessor, LlavaOnevisionForConditionalGeneration

model_name = "NCSOFT/VARCO-VISION-2.0-1.7B-OCR"
model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    attn_implementation="sdpa",
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_name)

image = Image.open("image.jpg")

`torch_dtype` is deprecated! Use `dtype` instead!


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
# Image upscaling for OCR performance boost
w, h = image.size
target_size = 2304
if max(w, h) < target_size:
    scaling_factor = target_size / max(w, h)
    new_w = int(w * scaling_factor)
    new_h = int(h * scaling_factor)
    image = image.resize((new_w, new_h))

conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": ""},
        ],
    },
]

inputs = processor.apply_chat_template(
    conversation,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(model.device, torch.float16)

In [3]:
generate_ids = model.generate(**inputs, max_new_tokens=1024)
generate_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generate_ids)
]
output = processor.decode(generate_ids_trimmed[0], skip_special_tokens=False)
print(output)

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


· 제품명제크 · 식품의 유형: 과자(유처리제품) · 롯데제과주식회사 경남 양산시 양산대로 1158 · 직사광선 및
· 영양성분 1회 제공량 1봉 (50g) 습기를 피해 서늘한 곳에 진열하시고, 유통중 변질품은 구입상점 및 본사에서 항상 교환해 드립니다. · 본 제품은
· 총 2회 제공량 (100g) 소비자기본법에 의한 소비자분쟁해결기준에 의거 정당한 소비자 피해에 대하여 교환·환불해 드립니다. · 휴지줍는 고운마을.
· 안버리는 밝은마음 · 유통기한한측면표기일까지(연.월.일) · 내포장제작폴리프로필렌 · 원재료명 밀가루(밀,미국산), 열량
· 탄수화물 33g 10% 가공유지(팜분별유(부분경화유):말레이시아산), 쇼트닝 I (정제가공유지(부분경화유), 당류 4g
· 말레이시아산), 쇼트닝 I (백설탕,산도조절제,액상과당,천일염,쇼트닝 I,맥아분말,치조분말, 단백질 3g 5%
· 레시틴,유청단백분말,효소제,합성착향료(브레드향), 밀,우유,대두,쇠고기,함유
· 포화지방 6g 40% 과자 표면에 있을 수 있는 짧은 실 모양은 맥아껍질이므로 안심하고 종이
· 트랜스지방 0.5g 0% 드시기 바랍니다 · 부정,불량식품 신고는 국번없이 1399
· 콜레스테롤 0mg 0% 콜레스테롤 400mg 20%
· 나트륨 080-024-6060 0% 영양성분기준치:1일 영양성분기준치에 대한 비율
· 수신자요율부담 8 80 1062 52 1906 1400
· 희망소비자가격 http://www.lottecont.co.kr
· 080-024-6060 0% 영양성분기준치:1일 영양성분기준치에 대한 비율<|im_end|>
